In [1]:
## Polecenia do instalacji pakietów
# conda install -c conda-forge tensorflow
import tensorflow as tf
import pandas as pd
import numpy as np

In [2]:
data_AUG = pd.read_csv("apartments_pl_2023_08.csv")
data_SEP = pd.read_csv("apartments_pl_2023_09.csv")
data_OCT = pd.read_csv("apartments_pl_2023_10.csv")

In [3]:
print("data_AUG length: ", len(data_AUG))
print("data_SEP length: ", len(data_SEP))
print("data_OCT length: ", len(data_OCT))

data_AUG length:  18905
data_SEP length:  16997
data_OCT length:  16690


# Przygotowanie danych

In [4]:
data_AUG.head()

,id,city,type,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,...,pharmacyDistance,ownership,buildingMaterial,condition,hasParkingSpace,hasBalcony,hasElevator,hasSecurity,hasStorageRoom,price
0,f8524536d4b09a0c8ccc0197ec9d7bde,szczecin,blockOfFlats,63.00,3.0,4.0,10.0,1980.0,53.378933,14.625296,...,0.413,condominium,concreteSlab,NaN,yes,yes,yes,no,yes,415000
1,accbe77d4b360fea9735f138a50608dd,szczecin,blockOfFlats,36.00,2.0,8.0,10.0,NaN,53.442692,14.559690,...,0.205,cooperative,concreteSlab,NaN,no,yes,yes,no,yes,395995
2,8373aa373dbc3fe7ca3b7434166b8766,szczecin,tenement,73.02,3.0,2.0,3.0,NaN,53.452222,14.553333,...,0.280,condominium,brick,NaN,no,no,no,no,no,565000
3,0a68cd14c44ec5140143ece75d739535,szczecin,tenement,87.60,3.0,2.0,3.0,NaN,53.435100,14.532900,...,0.087,condominium,brick,NaN,yes,yes,no,no,yes,640000
4,f66320e153c2441edc0fe293b54c8aeb,szczecin,blockOfFlats,66.00,3.0,1.0,3.0,NaN,53.410278,14.503611,...,0.514,condominium,NaN,NaN,no,no,no,no,no,759000


In [5]:
data_SEP.head()

,id,city,type,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,...,pharmacyDistance,ownership,buildingMaterial,condition,hasParkingSpace,hasBalcony,hasElevator,hasSecurity,hasStorageRoom,price
0,71de9eb8a92ac12d7c94315228b70cfb,szczecin,tenement,123.45,4.0,2.0,3.0,NaN,53.432586,14.535685,...,0.321,condominium,brick,NaN,no,yes,no,no,yes,799000
1,23d558163bb1c7863c73485048e50858,szczecin,NaN,53.80,2.0,5.0,5.0,NaN,53.429429,14.554642,...,0.073,cooperative,NaN,NaN,no,no,yes,no,no,420000
2,3b8819c6c9ffd55adfddfd5bafcc844d,szczecin,tenement,50.00,2.0,NaN,1.0,NaN,53.364400,14.660900,...,0.399,condominium,brick,NaN,yes,no,no,no,no,350000
3,708ccd264180315a4cfe6324ed4861d0,szczecin,blockOfFlats,55.19,2.0,1.0,2.0,2013.0,53.480364,14.541494,...,0.183,condominium,brick,premium,yes,yes,no,yes,yes,607000
4,8373aa373dbc3fe7ca3b7434166b8766,szczecin,tenement,73.02,3.0,2.0,3.0,NaN,53.452222,14.553333,...,0.280,condominium,brick,NaN,no,no,no,no,no,565000


In [6]:
data_OCT.head()

,id,city,type,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,...,pharmacyDistance,ownership,buildingMaterial,condition,hasParkingSpace,hasBalcony,hasElevator,hasSecurity,hasStorageRoom,price
0,23d558163bb1c7863c73485048e50858,szczecin,NaN,53.80,2.0,5.0,5.0,NaN,53.429429,14.554642,...,0.073,cooperative,NaN,NaN,no,no,yes,no,no,420000
1,8373aa373dbc3fe7ca3b7434166b8766,szczecin,tenement,73.02,3.0,2.0,3.0,NaN,53.452222,14.553333,...,0.280,condominium,brick,NaN,no,no,no,no,no,565000
2,ede1c0d953ff85f3cd9676d39ca8f62d,szczecin,blockOfFlats,37.00,2.0,8.0,10.0,1970.0,53.442692,14.559690,...,0.205,cooperative,concreteSlab,NaN,no,yes,yes,yes,yes,429995
3,6c807c6c86a74c57a3fd4445a1f49a46,szczecin,apartmentBuilding,48.00,2.0,4.0,6.0,NaN,53.435461,14.570436,...,0.177,condominium,brick,NaN,no,yes,no,no,no,550000
4,bb14c9a540275da73f6cb6ed42e5c26c,szczecin,apartmentBuilding,55.19,2.0,1.0,2.0,2013.0,53.482094,14.542574,...,0.043,condominium,brick,premium,yes,yes,no,yes,yes,607000


In [7]:
data_SEP.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16997 entries, 0 to 16996
Data columns (total 28 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    16997 non-null  object 
 1   city                  16997 non-null  object 
 2   type                  13171 non-null  object 
 3   squareMeters          16997 non-null  float64
 4   rooms                 16997 non-null  float64
 5   floor                 13954 non-null  float64
 6   floorCount            16751 non-null  float64
 7   buildYear             14048 non-null  float64
 8   latitude              16997 non-null  float64
 9   longitude             16997 non-null  float64
 10  centreDistance        16997 non-null  float64
 11  poiCount              16997 non-null  float64
 12  schoolDistance        16982 non-null  float64
 13  clinicDistance        16918 non-null  float64
 14  postOfficeDistance    16974 non-null  float64
 15  kindergartenDistanc

In [8]:
data_SEP.describe()

,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,centreDistance,poiCount,schoolDistance,clinicDistance,postOfficeDistance,kindergartenDistance,restaurantDistance,collegeDistance,pharmacyDistance,price
count,16997.000000,16997.000000,13954.000000,16751.000000,14048.000000,16997.000000,16997.000000,16997.000000,16997.000000,16982.000000,16918.000000,16974.000000,16983.000000,16954.000000,16495.000000,16974.000000,1.699700e+04
mean,59.033912,2.689357,3.327576,5.237538,1984.351936,52.066227,19.528680,4.347108,20.345590,0.418712,0.988139,0.524299,0.378864,0.359300,1.451509,0.370333,7.080972e+05
std,21.362743,0.916103,2.532150,3.305643,34.041196,1.331268,1.790088,2.893583,23.710083,0.482618,0.911026,0.516412,0.467429,0.487905,1.109576,0.483060,3.519888e+05
min,25.000000,1.000000,1.000000,1.000000,1851.000000,49.979110,14.447127,0.020000,0.000000,0.004000,0.005000,0.003000,0.002000,0.001000,0.007000,0.001000,1.549000e+05
25%,44.500000,2.000000,2.000000,3.000000,1965.000000,51.120355,18.529520,1.960000,6.000000,0.175000,0.359000,0.237000,0.158000,0.116000,0.583000,0.146000,4.650000e+05
50%,55.190000,3.000000,3.000000,4.000000,1991.000000,52.197569,19.903611,3.930000,13.000000,0.292000,0.681000,0.391000,0.268000,0.234000,1.122000,0.240000,6.390000e+05
75%,69.000000,3.000000,4.000000,6.000000,2014.000000,52.450200,20.998580,6.150000,24.000000,0.469000,1.266000,0.629000,0.423000,0.418000,2.077500,0.411000,8.500000e+05
max,150.000000,6.000000,29.000000,29.000000,2023.000000,54.606460,23.207128,16.920000,208.000000,4.818000,4.989000,4.905000,4.960000,4.985000,5.000000,4.861000,2.500000e+06


In [9]:
data_SEP.columns.values

array(['id', 'city', 'type', 'squareMeters', 'rooms', 'floor',
       'floorCount', 'buildYear', 'latitude', 'longitude',
       'centreDistance', 'poiCount', 'schoolDistance', 'clinicDistance',
       'postOfficeDistance', 'kindergartenDistance', 'restaurantDistance',
       'collegeDistance', 'pharmacyDistance', 'ownership',
       'buildingMaterial', 'condition', 'hasParkingSpace', 'hasBalcony',
       'hasElevator', 'hasSecurity', 'hasStorageRoom', 'price'],
      dtype=object)

In [10]:
print("Unique values for each column: ")
for column in data_SEP.columns.values:
    print("\t", len(data_SEP[column].unique()), "\t", column)

Unique values for each column: 
	 16997 	 id
	 15 	 city
	 4 	 type
	 4042 	 squareMeters
	 6 	 rooms
	 24 	 floor
	 30 	 floorCount
	 154 	 buildYear
	 12493 	 latitude
	 12695 	 longitude
	 1281 	 centreDistance
	 168 	 poiCount
	 1602 	 schoolDistance
	 3090 	 clinicDistance
	 1810 	 postOfficeDistance
	 1450 	 kindergartenDistance
	 1489 	 restaurantDistance
	 3786 	 collegeDistance
	 1485 	 pharmacyDistance
	 2 	 ownership
	 3 	 buildingMaterial
	 3 	 condition
	 2 	 hasParkingSpace
	 2 	 hasBalcony
	 3 	 hasElevator
	 2 	 hasSecurity
	 2 	 hasStorageRoom
	 2555 	 price


In [11]:
print("N/A values for each column: ")
for column in data_SEP.columns.values:
    print("\t", data_SEP[column].isna().sum(), "\t", column)

N/A values for each column: 
	 0 	 id
	 0 	 city
	 3826 	 type
	 0 	 squareMeters
	 0 	 rooms
	 3043 	 floor
	 246 	 floorCount
	 2949 	 buildYear
	 0 	 latitude
	 0 	 longitude
	 0 	 centreDistance
	 0 	 poiCount
	 15 	 schoolDistance
	 79 	 clinicDistance
	 23 	 postOfficeDistance
	 14 	 kindergartenDistance
	 43 	 restaurantDistance
	 502 	 collegeDistance
	 23 	 pharmacyDistance
	 0 	 ownership
	 6635 	 buildingMaterial
	 12958 	 condition
	 0 	 hasParkingSpace
	 0 	 hasBalcony
	 868 	 hasElevator
	 0 	 hasSecurity
	 0 	 hasStorageRoom
	 0 	 price


In [14]:
# dodanie miesiąca, w którym dana oferta sprzedaży była dostępna (wszystkie dane pochodzą z roku 2023)
data_AUG["offerMonth"] = 8
data_SEP["offerMonth"] = 9
data_OCT["offerMonth"] = 10

In [15]:
data_ALL = pd.concat([data_AUG, data_SEP, data_OCT], ignore_index=True)

In [16]:
data_ALL.tail()

,id,city,type,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,...,ownership,buildingMaterial,condition,hasParkingSpace,hasBalcony,hasElevator,hasSecurity,hasStorageRoom,price,offerMonth
52587,1e7f4f1fdfea31eb84e071d697839632,bydgoszcz,NaN,50.12,2.0,1.0,1.0,NaN,53.129657,18.003888,...,condominium,brick,NaN,yes,no,no,no,no,360000,10
52588,d971c4ed3aab9919bf203e96019a678e,bydgoszcz,tenement,100.00,4.0,3.0,4.0,1910.0,53.126000,18.007900,...,condominium,brick,premium,yes,yes,no,no,yes,590000,10
52589,60fcbfa2a2a48ebcc2e554efba7f2729,bydgoszcz,tenement,59.10,2.0,1.0,1.0,1910.0,53.126000,18.007900,...,condominium,brick,NaN,yes,no,no,no,yes,320000,10
52590,cd0241b70b79aaaf767a0dd3a7cfbb31,bydgoszcz,NaN,81.07,5.0,1.0,4.0,2019.0,53.115921,17.956396,...,condominium,NaN,NaN,no,yes,yes,no,no,679000,10
52591,1cdf62d567c6be2fa488f16ad9939c3a,bydgoszcz,blockOfFlats,133.16,4.0,3.0,3.0,NaN,53.112086,17.989945,...,condominium,brick,premium,no,yes,no,no,yes,925000,10


In [17]:
data_ALL.count()

id                      52592
city                    52592
type                    40942
squareMeters            52592
rooms                   52592
floor                   43065
floorCount              51842
buildYear               43467
latitude                52592
longitude               52592
centreDistance          52592
poiCount                52592
schoolDistance          52542
clinicDistance          52340
postOfficeDistance      52519
kindergartenDistance    52543
restaurantDistance      52438
collegeDistance         51020
pharmacyDistance        52513
ownership               52592
buildingMaterial        32087
condition               12773
hasParkingSpace         52592
hasBalcony              52592
hasElevator             49904
hasSecurity             52592
hasStorageRoom          52592
price                   52592
offerMonth              52592
dtype: int64

In [28]:
# porównanie duplikatów, z pominięciem dodanej kolumny offerMonth
data_ALL.loc[data_ALL.duplicated(data_ALL.columns.delete(data_ALL.columns.get_loc("offerMonth")))]

,id,city,type,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,...,ownership,buildingMaterial,condition,hasParkingSpace,hasBalcony,hasElevator,hasSecurity,hasStorageRoom,price,offerMonth
18905,71de9eb8a92ac12d7c94315228b70cfb,szczecin,tenement,123.45,4.0,2.0,3.0,NaN,53.432586,14.535685,...,condominium,brick,NaN,no,yes,no,no,yes,799000,9
18906,23d558163bb1c7863c73485048e50858,szczecin,NaN,53.80,2.0,5.0,5.0,NaN,53.429429,14.554642,...,cooperative,NaN,NaN,no,no,yes,no,no,420000,9
18909,8373aa373dbc3fe7ca3b7434166b8766,szczecin,tenement,73.02,3.0,2.0,3.0,NaN,53.452222,14.553333,...,condominium,brick,NaN,no,no,no,no,no,565000,9
18912,eb15c0a8a7f16a523f5cf5f61de06638,szczecin,blockOfFlats,49.50,2.0,1.0,4.0,1980.0,53.365833,14.661389,...,cooperative,NaN,low,no,yes,no,no,yes,462000,9
18922,209a6d2dea99216bdf83a968343cd3b3,szczecin,tenement,61.48,2.0,NaN,3.0,1900.0,53.491508,14.606985,...,condominium,brick,premium,no,no,no,no,yes,390000,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52587,1e7f4f1fdfea31eb84e071d697839632,bydgoszcz,NaN,50.12,2.0,1.0,1.0,NaN,53.129657,18.003888,...,condominium,brick,NaN,yes,no,no,no,no,360000,10
52588,d971c4ed3aab9919bf203e96019a678e,bydgoszcz,tenement,100.00,4.0,3.0,4.0,1910.0,53.126000,18.007900,...,condominium,brick,premium,yes,yes,no,no,yes,590000,10
52589,60fcbfa2a2a48ebcc2e554efba7f2729,bydgoszcz,tenement,59.10,2.0,1.0,1.0,1910.0,53.126000,18.007900,...,condominium,brick,NaN,yes,no,no,no,yes,320000,10
52590,cd0241b70b79aaaf767a0dd3a7cfbb31,bydgoszcz,NaN,81.07,5.0,1.0,4.0,2019.0,53.115921,17.956396,...,condominium,NaN,NaN,no,yes,yes,no,no,679000,10


In [31]:
# w poszczególnych miesiącach nie występują duplikaty tej samej oferty, ponieważ metoda duplicated() 
# nie znajduje wierszy jeśli uwzględniona zostanie kolumna "offerMonth"
data_ALL.loc[data_ALL.duplicated()]

,id,city,type,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,...,ownership,buildingMaterial,condition,hasParkingSpace,hasBalcony,hasElevator,hasSecurity,hasStorageRoom,price,offerMonth


In [32]:
data_FILTERED = data_ALL.drop_duplicates(subset=["id"], keep='last')
data_FILTERED

,id,city,type,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,...,ownership,buildingMaterial,condition,hasParkingSpace,hasBalcony,hasElevator,hasSecurity,hasStorageRoom,price,offerMonth
0,f8524536d4b09a0c8ccc0197ec9d7bde,szczecin,blockOfFlats,63.00,3.0,4.0,10.0,1980.0,53.378933,14.625296,...,condominium,concreteSlab,NaN,yes,yes,yes,no,yes,415000,8
1,accbe77d4b360fea9735f138a50608dd,szczecin,blockOfFlats,36.00,2.0,8.0,10.0,NaN,53.442692,14.559690,...,cooperative,concreteSlab,NaN,no,yes,yes,no,yes,395995,8
4,f66320e153c2441edc0fe293b54c8aeb,szczecin,blockOfFlats,66.00,3.0,1.0,3.0,NaN,53.410278,14.503611,...,condominium,NaN,NaN,no,no,no,no,no,759000,8
6,ec27024bfcd012728617a35dad2cb6b8,szczecin,blockOfFlats,47.45,2.0,2.0,10.0,1974.0,53.450232,14.562625,...,condominium,concreteSlab,low,no,no,yes,no,yes,370000,8
8,7e1981e920d763d6237c5bdcf13cf5b7,szczecin,blockOfFlats,47.76,2.0,8.0,12.0,1980.0,53.458869,14.536403,...,condominium,concreteSlab,NaN,no,yes,yes,no,yes,399000,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52587,1e7f4f1fdfea31eb84e071d697839632,bydgoszcz,NaN,50.12,2.0,1.0,1.0,NaN,53.129657,18.003888,...,condominium,brick,NaN,yes,no,no,no,no,360000,10
52588,d971c4ed3aab9919bf203e96019a678e,bydgoszcz,tenement,100.00,4.0,3.0,4.0,1910.0,53.126000,18.007900,...,condominium,brick,premium,yes,yes,no,no,yes,590000,10
52589,60fcbfa2a2a48ebcc2e554efba7f2729,bydgoszcz,tenement,59.10,2.0,1.0,1.0,1910.0,53.126000,18.007900,...,condominium,brick,NaN,yes,no,no,no,yes,320000,10
52590,cd0241b70b79aaaf767a0dd3a7cfbb31,bydgoszcz,NaN,81.07,5.0,1.0,4.0,2019.0,53.115921,17.956396,...,condominium,NaN,NaN,no,yes,yes,no,no,679000,10


In [33]:
data_FILTERED.count()

id                      32027
city                    32027
type                    24844
squareMeters            32027
rooms                   32027
floor                   26314
floorCount              31609
buildYear               26478
latitude                32027
longitude               32027
centreDistance          32027
poiCount                32027
schoolDistance          31998
clinicDistance          31870
postOfficeDistance      31983
kindergartenDistance    31997
restaurantDistance      31928
collegeDistance         31081
pharmacyDistance        31977
ownership               32027
buildingMaterial        19065
condition                7732
hasParkingSpace         32027
hasBalcony              32027
hasElevator             30407
hasSecurity             32027
hasStorageRoom          32027
price                   32027
offerMonth              32027
dtype: int64

# Analiza danych

Wyniki analizy danych (specyficzne dla konkretnego zbioru danych) standardowo powinny zawierać przynajmniej następujące elementy:

    liczba atrybutów i typy ich wartości
    liczba różnych wartości atrybutów, które nie są ciągłe (ze skończonego, niewielkiego zakresu)
    atrybut decyzyjny - charakterystyka
    brakujące wartości (z podziałem na obiekty i w ramach poszczególnych atrybutów warunkowych)
    duplikaty
    liczebności przykładów z poszczególnych klas
    współczynnik IR (imbalanced ratio)
    mapa korelacji atrybutów
    weryfikacja zasadności pozostawienia/usunięcia poszczególnych atrybutów na podstawie wiedzy dziedzinowej
    analiza (komentarze dodawane na bieżąco w notatniku)
    wnioski

## liczba atrybutów i typy ich wartości

## liczba różnych wartości atrybutów, które nie są ciągłe (ze skończonego, niewielkiego zakresu)

## atrybut decyzyjny - charakterystyka

## brakujące wartości (z podziałem na obiekty i w ramach poszczególnych atrybutów warunkowych)

## duplikaty

## liczebności przykładów z poszczególnych klas

## współczynnik IR (imbalanced ratio)

## mapa korelacji atrybutów

## weryfikacja zasadności pozostawienia/usunięcia poszczególnych atrybutów na podstawie wiedzy dziedzinowej

## wnioski